In [ ]:
# default_exp core

# ihu

> Evolve any unitary using solve_ivp.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev.export import notebook2script
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
from scipy.integrate import solve_ivp
from numpy.random import random_sample
from numpy import array, linspace
from math import sqrt
from ihu.herm import rand_herm_ndarray, test_herm_ndarray
from ihu.unitary import rand_unitary_ndarray, test_unitary_ndarray

In [ ]:
#export
def ihu(ham_func, u0, T, no_time_step_for_eval, **kwargs):
    r"""evolves the unitary
        
    Parameters
    ----------
    ham_func : function
        Function which produces the Hamiltonian 
        as it appears at time t
        form ham_func(t)
    u0 : ndarray
        Unitary at time 0
    T : float
        Evolution time
    no_time_step_for_eval : int
        no of time steps for evaluation in the 
        final output
    **kwargs
        Arbitrary optional keyword arguments.
        w : float
            Defaults to 6.28
    
    Returns
    -------
    Bunch object with the following fields defined:
    t  : [ndarray, shape (n_points,)] 
        Time points.        
    y  : [ndarray, shape (n, n_points)] 
        Values of the solution at t.
    sol: [OdeSolution or None] 
        Found solution as OdeSolution instance; None if
        dense_output was set to False.
    t_events: [list of ndarray or None]
        Contains for each event type a list of arrays at which an event of
        that type event was detected. None if events was None.
    nfev: [int] 
        Number of evaluations of the right-hand side.
    njev: [int]
        Number of evaluations of the Jacobian.
    nlu :[int] 
        Number of LU decompositions.
    status [int] 
        Reason for algorithm termination:
        • -1: Integration step failed.
        • 0: The solver successfully reached the end of tspan.
        • 1: A termination event occurred.
    message: [string] 
        Human-readable description of the termination reason.
    success: [bool] 
        True if the solver reached the interval end or a 
        termination event occurred (status
    >= 0).
    
    u_series :[ndarray, shape (sqrt(n), sqrt(n), n_points)] 
        Values of the solution at t.
    
    """
    u_shape = u0.shape()
    
    check_for_stupidity(ham_func, u0, )
    T = evolution_time
    Nt = no_time_step_for_eval
    dt = T/Nt
    y0 = u0.ravel()
    t_eval_span = linspace(0, T, Nt)
    
    result = solve_ivp(fun=y_der, t_span=(0, T), y0=y0, 
                       t_eval=t_eval_span, args=(u_shape, ham_func))
    # Will expose other options later on
    
    print(result.message)
    
    u_series = result.y.reshape(u_shape, u_shape, result.y.shape[-1])

    return u_series, result


Steps:
   - Determines Shape of U0.
   - Determines Shape of H (at any random time t).
   - If anyone of U, H are non square exit.
   - If shapes match proceed else exit.
   - make sure Output of H is Hermitian 
      (by trying it out at 3 different times)
   - Initialize inputs T, Nt
   - Flatten u0 to get y0
   - calculate dt = T/ Nt
   - make y_der
   - evolve y
   - collect results
   - y_series to u_series
   - print messages
   - return stuff + u_series
        


In [ ]:
linspace(1, 10, 10)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [ ]:
linspace(1, 10, 100)

array([ 1.        ,  1.09090909,  1.18181818,  1.27272727,  1.36363636,
        1.45454545,  1.54545455,  1.63636364,  1.72727273,  1.81818182,
        1.90909091,  2.        ,  2.09090909,  2.18181818,  2.27272727,
        2.36363636,  2.45454545,  2.54545455,  2.63636364,  2.72727273,
        2.81818182,  2.90909091,  3.        ,  3.09090909,  3.18181818,
        3.27272727,  3.36363636,  3.45454545,  3.54545455,  3.63636364,
        3.72727273,  3.81818182,  3.90909091,  4.        ,  4.09090909,
        4.18181818,  4.27272727,  4.36363636,  4.45454545,  4.54545455,
        4.63636364,  4.72727273,  4.81818182,  4.90909091,  5.        ,
        5.09090909,  5.18181818,  5.27272727,  5.36363636,  5.45454545,
        5.54545455,  5.63636364,  5.72727273,  5.81818182,  5.90909091,
        6.        ,  6.09090909,  6.18181818,  6.27272727,  6.36363636,
        6.45454545,  6.54545455,  6.63636364,  6.72727273,  6.81818182,
        6.90909091,  7.        ,  7.09090909,  7.18181818,  7.27

In [ ]:
#export
def y_der(t, y, u_shape, ham_func):
    r"""Basically dy/dt
    
    Does some stuff.
    
    Parameters
    ----------
    t : float
        Current time value during the integration
    y : ndarray
        flattened unitary at the current time
    
    Returns
    -------
    dydt : ndarray
        flattened derivative of the unitary
    
    """
    # u_shape = int(sqrt(y.shape(0)))
    u_curr = y.reshape(u_shape, u_shape)
    hamiltonian_curr = ham_func(t)
    dudt = -1j*hamiltonian_curr@u_curr
    dydt = dudt.ravel()
    
    return dydt

In [ ]:
#export
def check_for_stupidity(ham_func, u0, ):
    r"""Perform checks for invalid inputs and the lot"""
    u0_shape = u0.shape()
    ham_shape = ham_func(0.2).shape()
    
    
    try:
        assert u0_shape[0] == u0_shape[1]
    except AssertionError as e1:
        print('Initial unitary is not a square matrix')
        raise AssertionError
        
    try:
        assert ham_shape[0] == ham_shape[1]
    except AssertionError as e2:
        print('Hamiltonian is not a square matrix') 
        raise AssertionError
    
    try:
        assert u0_shape == ham_shape
    except AssertionError as e3:
        print('Hamiltonian shape is not same as that of the initial unitary.')
        raise AssertionError
    
    try:
        assert test_unitary_ndarray(u0) is True
    except AssertionError as e4::
        print('Initial unitary is not unitary')
        raise AssertionError
        
    try:
        assert test_herm_ndarray(ham_func(0.42)) is True
    except AssertionError as e4::
        print('Hamiltonian is not Hermitian')
        raise AssertionError
        
    
    
    return None

In [ ]:
    try:
        assert test_herm_ndarray(u0) is True
    except AssertionError as e4::
        print('Initial herm is not herm')
        raise AssertionError

In [ ]:
sqrt(9)

3.0

In [ ]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_reshape.ipynb.
Converted 02_herm.ipynb.
Converted 03_unitary.ipynb.
Converted index.ipynb.
